In [1]:
import numpy as np

hydration_after_p = 0.8
hydration_before_p = 0.045
#Wet Weight(g) after pretreatment
#2% 1h, 12% 1h, 2% 3h 12% 3h
w_60c= np.array([7.8302,4.0316,5.45,4.48]) 
w_80c= np.array([6.4508, 5.181, 5.929, 3.9756]) 
w_100c = np.array([5.6054, 4.1634, 5.9361, 3.9318]) 
w_120c = np.array([4.8918, 3.3319, 5.1648, 2.9348]) 

#Dry Weight(g) after pretreatment
dry_w_60c = w_60c * (1-hydration_after_p)
dry_w_80c = w_80c * (1-hydration_after_p)
dry_w_100c = w_100c * (1-hydration_after_p)
dry_w_120c = w_120c * (1-hydration_after_p)

avg_lost_w_60c = dry_w_60c/2
avg_lost_w_80c = dry_w_80c/2
avg_lost_w_100c = dry_w_100c/2
avg_lost_w_120c = dry_w_120c/2

dry_w_60c, dry_w_80c, dry_w_100c, dry_w_120c

(array([1.56604, 0.80632, 1.09   , 0.896  ]),
 array([1.29016, 1.0362 , 1.1858 , 0.79512]),
 array([1.12108, 0.83268, 1.18722, 0.78636]),
 array([0.97836, 0.66638, 1.03296, 0.58696]))

In [2]:
def calculate_dry_weight(w, hydration_after_p):
    return w * (1 - hydration_after_p)

def calculate_hplc(hplc, k, m, dilution_factor=5):
    return np.array([dilution_factor*(value-m)/k for value in hplc])

def calculate_mass(c, volume_H):
    return c * volume_H

def calculate_max(values, dry_masses):
    return [values[i+1]*value for i, value in enumerate(dry_masses)]

def calculate_yield(m, max_m):
    return np.divide(m, max_m)

def calculate_average(pairs):
    return np.mean(pairs, axis=1)

def reshape_into_pairs(array):
    return array.reshape(-1, 2)

In [ ]:
def preprocess_data(G_samples_hplc, XMG_samples_hplc, glucose_values, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment):

    pairs_G_C = reshape_into_pairs(G_samples_hplc)
    pairs_XMG_C = reshape_into_pairs(XMG_samples_hplc)

    avgs_G_C = calculate_average(pairs_G_C)
    avgs_XMG_C = calculate_average(pairs_XMG_C)
  
    c_G = calculate_hplc(avgs_G_C, k=315718, m=4623)
    c_XMG = calculate_hplc(avgs_XMG_C, k=317400, m=10216)

    m_G = calculate_mass(c_G, volume)
    m_XMG = calculate_mass(c_XMG, volume)

    #Over the pretreatment
    max_glucose_p_grams = calculate_max(glucose_values, dry_masses)
    max_XMG_p_grams = calculate_max(xylmangal_values, dry_masses)
    yield_glucose_p = calculate_yield(m_G, max_glucose_p_grams)
    yield_XMG_p = calculate_yield(m_XMG, max_XMG_p)

    #Over the process
    yield_glucose_r = calculate_yield(m_G, max_glucose_r)
    yield_XMG_r = calculate_yield(m_XMG, max_XMG_r)
    max_glucose_r = glucose_values[0]
    max_XMG_r = xylmangal_values[0]
    monosacharides = np.add(glucose_values, xylmangal_values)
    max_monosacharides_p = calculate_max(monosacharides, dry_masses)
    max_monosacharides_r = monosacharides[0]

    yield_monosacharides_p = calculate_yield(m_G + m_XMG, max_monosacharides_p)
    yield_monosacharides_r = calculate_yield(m_G + m_XMG, max_monosacharides_r)

    results = {
        "Milled mass": average_dry_masses_before_pretreatment,

        "Max Yield Glucose over Process": 39.54,
        "Max Yield XMG over Process": 
        "mass_XMG": m_XMG,
        "mass_Glucose": m_G,
        "max_glucose_%": max_glucose_r,
        "max_XMG_%": max_XMG_r,
        "yield_glucose_p": yield_glucose_p,
        "yield_XMG_p": yield_XMG_p,
        "yield_glucose_r": yield_glucose_r,
        "yield_XMG_r": yield_XMG_r,
        "yield_monosacharides_p": yield_monosacharides_p,
        "yield_monosacharides_r": yield_monosacharides_r
    }

    return results

In [3]:
#Varmast 1h, varmast 3h, kallast 1h, kallast 3h
#Ex 120C 1h, 120C 3h, 60C 1h, 60C 3h

#CHANGE THESE
milled_masses = np.array([2.0924,2.0960,2.0954,2.0985,2.0923,2.0979,2.0962,2.0943])
Glucose_samples_hplc = np.array([590520.1774,  626867.5736, 693851.4441, 661331.5045, 578989.8003, 675418.0375, 571507.7927, 596580.4333])
XMG_samples_hplc = np.array([149706.8617, 159126.8929,  176542.8571,  168805.7143,  147914.2857,  172857.1429,  146857.1429,  153200.0000])

#Constants
glucose_values = np.array([39.54, 88.30, 68.42, 65.11, 56.28])/100
xylmangal_values = np.array([25.24, 2.64, 1.97, 9.96, 7.4])/100
volume = 0.02485
hydration_after_p = 0.8

dry_masses_before_p = milled_masses*0.955
average_dry_masses_before_pretreatment = calculate_average(reshape_into_pairs(dry_masses_before_p))
a = average_dry_masses_before_pretreatment
print(f"Average dry masses before pretreatment: {a}")

#CHANGE THIS 
dry_masses_after_p = np.array([a[0]*avg_lost_w_120c[0], a[1]*avg_lost_w_120c[2], a[2]*avg_lost_w_60c[0], a[3]*avg_lost_w_60c[2]])

print(f"dryu masses after p: {dry_masses_after_p}")

Average dry masses before pretreatment: [1.999961   2.00258725 2.0008205  2.00096375]
dryu masses after p: [0.97834092 1.03429626 1.56668247 1.09052524]


In [4]:
data = preprocess_data(Glucose_samples_hplc, XMG_samples_hplc, glucose_values, xylmangal_values, volume, dry_masses_after_p, average_dry_masses_before_pretreatment)
import pandas as pd

index = ['120h 1h', '120h 3h', '60h 1h', '60h 3h']
df = pd.DataFrame(data)
df.index = index
print('2% NaOH batch')
print(df)

NameError: name 'preprocess_data' is not defined